# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import pandas as pd
import numpy as np 
from sklearn import datasets
import joblib

from azureml.core import Experiment, Model, Environment, ScriptRunConfig
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace, Dataset, Datastore 
from azureml.pipeline.steps import AutoMLStep
from azureml.core.model import InferenceConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [28]:
ws = Workspace.from_config()
experiment_name = 'heart-failure-pred'

ws = Workspace.from_config()
ws.write_config(path='.azureml')
exp = Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()


Workspace name: quick-starts-ws-158470
Azure region: southcentralus
Subscription id: 5a4ab2ba-6c51-4805-8155-58759ad589d8
Resource group: aml-quickstarts-158470


In [29]:
datastore_name = 'workspaceblobstore'
datastore = Datastore.get(ws, datastore_name)
datastore_path = [(datastore, 'UI/09-16-2021_114439_UTC/heart_failure_clinical_records_dataset.csv')]
ds = Dataset.Tabular.from_delimited_files(path=datastore_path)
ds = ds.take(10).to_pandas_dataframe()

In [31]:
from azureml.core.compute_target import ComputeTargetException


amlcompute_cluster_name = "project-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           min_nodes=1,    
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [36]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
import os 
import shutil 

early_termination_policy = BanditPolicy(slack_factor=0.1, evaluation_interval=2)

if "training" not in os.listdir():
    os.mkdir("./training")
script_folder = './training'
os.makedirs(script_folder, exist_ok=True)
shutil.copy('./train.py', script_folder)    

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
                                      "--C": uniform(0.1,1),
                                      "max_iter": choice([50, 75, 100, 125])
})
#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory=script_folder,compute_target="project-cluster", entry_script='train.py')

hyperdrive_run_config = HyperDriveConfig(
    hyperparameter_sampling=param_sampling,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=4,
    estimator=estimator,
    policy=early_termination_policy

)

In [37]:
#TODO: Submit your experiment
hyperdrive_run = exp.submit(config=hyperdrive_run_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_8cd1e37a-7095-41ce-909d-5ed3e631929a
Web View: https://ml.azure.com/runs/HD_8cd1e37a-7095-41ce-909d-5ed3e631929a?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-158470/workspaces/quick-starts-ws-158470&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-09-16T14:24:00.454439][API][INFO]Experiment created<END>\n""<START>[2021-09-16T14:24:00.915681][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-09-16T14:24:01.222152][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_8cd1e37a-7095-41ce-909d-5ed3e631929a
Web View: https://ml.azure.com/runs/HD_8cd1e37a-7095-41ce-909d-5ed3e631929a?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-158470/workspaces/quick-starts-ws-158470&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Warnings:
{
  "error

{'runId': 'HD_8cd1e37a-7095-41ce-909d-5ed3e631929a',
 'target': 'project-cluster',
 'status': 'Canceled',
 'startTimeUtc': '2021-09-16T14:24:00.158911Z',
 'endTimeUtc': '2021-09-16T14:27:03.420403Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null,\n    "additionalInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'prim

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [25]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_31b20807-448a-421b-bbc4-0c6bfc6e061b
Web View: https://ml.azure.com/runs/HD_31b20807-448a-421b-bbc4-0c6bfc6e061b?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-158470/workspaces/quick-starts-ws-158470&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Execution Summary
RunId: HD_31b20807-448a-421b-bbc4-0c6bfc6e061b
Web View: https://ml.azure.com/runs/HD_31b20807-448a-421b-bbc4-0c6bfc6e061b?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-158470/workspaces/quick-starts-ws-158470&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Warnings:
{
  "error": {
    "code": "UserError",
    "severity": null,
    "message": "User errors were found in at least one of the child runs.",
    "messageFormat": null,
    "messageParameters": {},
    "referenceCode": null,
    "detailsUri": null,
    "target": null,
    "details": [],
    "innerError": null,
    "debugInfo": null,
    "additionalInfo": null
  },
  "correlation": null

{'runId': 'HD_31b20807-448a-421b-bbc4-0c6bfc6e061b',
 'target': 'project-cluster',
 'status': 'Canceled',
 'startTimeUtc': '2021-09-16T13:41:52.035557Z',
 'endTimeUtc': '2021-09-16T13:44:58.810732Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null,\n    "additionalInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'prim

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service